In [1]:

import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-10-31 18:05:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.37MB/s    in 0.7s    

2020-10-31 18:05:37 (1.37 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [5]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import *
# Read in the Review dataset as a DataFrame

In [6]:
Imported_vine_votes20_df = df.filter('total_votes >19')

In [7]:
Imported_vine_votes20_df.count()

72587

In [8]:
temp_df = Imported_vine_votes20_df.withColumn('helpful_votes50',Imported_vine_votes20_df['helpful_votes'] / Imported_vine_votes20_df['total_votes'])
votes50_df = temp_df.filter('helpful_votes50 >= 0.50')
votes50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   44689470|R2WOW0TURNXB26|B00YY3UBV2|     310491927|            Garmin 1|        Wireless|          3|           54|         59|   N|                Y|Pretty Disappoint...|Bought this unit ...| 2015-08-31|0.9152542372881356|
|         US|     112342|R13VL62Y2HBQ0B|B010

In [9]:
votes_count50 = votes50_df.count()
votes_count50

65581

In [10]:
star5_df = votes50_df.filter('star_rating == "5"')
star5_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|     112342|R13VL62Y2HBQ0B|B010VFZJD6|     129632031|iTaste MVP3 PRO -...|        Wireless|          5|           15|         21|   N|                Y|   Excellent battery|I love my Mvp pro...| 2015-08-31|0.7142857142857143|
|         US|   22953177|R111DJA10Y6CMU|B013

In [11]:
star5_count = star5_df.count()
star5_count

30765

In [12]:
vine_only_df = votes50_df.filter('vine == "Y"')
vine_only_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   48852155|R1MAOLI5FJHAFM|B013X0V11K|     610966690|BLU Studio 7.0 ll...|        Wireless|          4|          249|        261|   Y|                N|More than just a ...|Ever since the ve...| 2015-08-31|0.9540229885057471|
|         US|   11556116| R9PYAUDIBJVEC|B013

In [13]:
vine_only_count= vine_only_df.count()
vine_only_count

613

In [14]:
vine_only_star5 = vine_only_df.filter('star_rating == "5"')
vine_only_star5.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   46671309| R6V9SHMMG5M8F|B013X0V11K|     610966690|BLU Studio 7.0 ll...|        Wireless|          5|          101|        110|   Y|                N|  Very good Phablet!|I remember the da...| 2015-08-30|0.9181818181818182|
|         US|   53019847|R3SRW1E8J56IGV|B012

In [15]:
vine_only_star5_count = vine_only_star5.count()
vine_only_star5_count

222

In [18]:
not_vine_only_df = votes50_df.filter('vine == "N"')
not_vine_only_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   44689470|R2WOW0TURNXB26|B00YY3UBV2|     310491927|            Garmin 1|        Wireless|          3|           54|         59|   N|                Y|Pretty Disappoint...|Bought this unit ...| 2015-08-31|0.9152542372881356|
|         US|     112342|R13VL62Y2HBQ0B|B010

In [19]:
not_vine_count = not_vine_only_df.count()
not_vine_count

64968

In [20]:
not_vine_only_5star_df = not_vine_only_df.filter('star_rating == "5"')
not_vine_only_5star_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|     112342|R13VL62Y2HBQ0B|B010VFZJD6|     129632031|iTaste MVP3 PRO -...|        Wireless|          5|           15|         21|   N|                Y|   Excellent battery|I love my Mvp pro...| 2015-08-31|0.7142857142857143|
|         US|   22953177|R111DJA10Y6CMU|B013

In [21]:
not_vine_5star_count = not_vine_only_5star_df.count()
not_vine_5star_count

30543

In [22]:
star5_vine_perc_all = (vine_only_star5_count *100/ votes_count50)
star5_vine_perc_all

0.33851267897714277

In [23]:
star5_not_vine_perc_all = (not_vine_5star_count *100/ votes_count50)
star5_not_vine_perc_all

46.57294033332825

In [24]:
star5_vine_perc_all5star = (vine_only_star5_count *100/ star5_count)
star5_vine_perc_all5star

0.7215992198927352

In [25]:
not_vine_5star_perc_all = (not_vine_5star_count*100 / star5_count )
not_vine_5star_perc_all

99.27840078010726

In [26]:
star5vine_all_vine_perc = (vine_only_star5_count *100/ vine_only_count)
star5vine_all_vine_perc

36.215334420880914

In [27]:
star5non_vine_all_nonvine_perc = (not_vine_5star_count *100/ not_vine_count)
star5non_vine_all_nonvine_perc

47.01237532323606

In [28]:
import pandas as pd
summary_counts = {'Count Title': ['Total Filtered Vote Counts','Total Five Star Vote Counts', 'Total Vine Votes Count','Total Vine Five Star Votes Count','Total Non-Vine Votes Count', 'Total Non-Vine Five Star Votes Count'],
                  'Counts': [votes_count50, star5_count, vine_only_count, vine_only_star5_count, not_vine_count,not_vine_5star_count]}
              
summary_counts_df = pd.DataFrame(summary_counts)
summary_counts_df


,Count Title,Counts
0,Total Filtered Vote Counts,65581
1,Total Five Star Vote Counts,30765
2,Total Vine Votes Count,613
3,Total Vine Five Star Votes Count,222
4,Total Non-Vine Votes Count,64968
5,Total Non-Vine Five Star Votes Count,30543


In [29]:
percentages = {'Percent Title' : ['% of 5 Star Vine Only(Paid) to All Filtered Votes', '% of 5 Star Non-Vine Only(Not Paid) to All Filtered Votes', '% of 5 Star Vine Only(Paid) to all 5 Star Votes', '% of 5 Star Non-Vine Only(Not Paid) to All 5 Star Votes', '% of Vine 5 Star to all Vine', '% of Non-Vine 5 star to All non-vine'],
              'Percentage' : [star5_vine_perc_all, star5_not_vine_perc_all, star5_vine_perc_all5star, not_vine_5star_perc_all, star5vine_all_vine_perc, star5non_vine_all_nonvine_perc ]}
percentages_df = pd.DataFrame(percentages)
percentages_df

,Percent Title,Percentage
0,% of 5 Star Vine Only(Paid) to All Filtered Votes,0.338513
1,% of 5 Star Non-Vine Only(Not Paid) to All Fil...,46.572940
2,% of 5 Star Vine Only(Paid) to all 5 Star Votes,0.721599
3,% of 5 Star Non-Vine Only(Not Paid) to All 5 S...,99.278401
4,% of Vine 5 Star to all Vine,36.215334
5,% of Non-Vine 5 star to All non-vine,47.012375


In [31]:
star5_df.toPandas().to_csv('vine.csv')

In [32]:
from google.colab import files
files.download("vine.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>